In [0]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os

In [0]:
# training parameters
batch_size=32
# original paper trained all networks with batch_size = 128
epochs = 200
data_augmentation = True
num_classes = 10

# substracting pixel mean improves accuracy
subtract_pixel_mean = True

n=3

# Model version
# original paper: version=1 (ResNet v1)
# Improved ResNet: version=2 (ResNet v2)
version = 1
if version == 1:
    depth = n*6 + 2
elif version == 2:
    depth = n*9 + 2

In [15]:
# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# load the CIFAR10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# input image dimensions
input_shape = x_train.shape[1:]

# normalize the data
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# if subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape: ', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape: ', y_train.shape)

x_train shape:  (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape:  (50000, 1)


In [0]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
def lr_schedule(epoch):
    # learning rate schedule
    # learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs
    # called automatically every epoch as part of callbacks during training
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [0]:
def resnet_layer(inputs, 
                num_filters = 16,
                kernel_size = 3,
                strides = 1,
                activation = 'relu',
                batch_normalization = True,
                conv_first=True):
    # 2D Convolution-Batch Normalization-Activation stack builder
    conv = Conv2D(num_filters,
                 kernel_size=kernel_size,
                 strides = strides,
                 padding='same',
                 kernel_initializer='he_normal',
                 kernel_regularizer=l2(1e-4))
    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [0]:
def resnet_v1(input_shape, depth, num_classes=10):
    # ResNet version 1 Model builder
    if (depth-2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # start model definition
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)
    
    inputs = Input(shape = input_shape)
    x = resnet_layer(inputs = inputs)
    # instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(3):
            strides = 1
            if stack > 0 and res_block == 0:
                # first layer but not first stack
                stides = 2
                # downsample
            y = resnet_layer(inputs=x,
                            num_filters = num_filters,
                            strides = strides)
            y = resnet_layer(inputs=x,
                            num_filters = num_filters,
                            activation = None)
            if stack > 0 and res_block == 0:
                # first layer but not first stack
                # linear projection residual shortcut connection to match
                # change dims
                x = resnet_layer(inputs = x,
                                num_filters = num_filters,
                                kernel_size = 1,
                                strides = strides,
                                activation = None,
                                batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2
    
    # add classifier on top
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                   activation = 'softmax',
                   kernel_initializer='he_normal')(y)
    
    # instantiate model
    model = Model(inputs = inputs, outputs = outputs)
    return model

In [0]:
def resnet_v2(input_shape, depth, num_classes=10):
    # ResNet version Model builder [b]
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2')
    # start model definition
    num_filters_in = 16
    num_res_blocks = int((depth - 2)/9)
    
    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_alyer(inputs = inputs,
                    num_filters=num_filters_in,
                    conv_first=True)
    
    # instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:
                    # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:
                    # first layer but not first stage
                    strides = 2
                    # downsample
            
            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                            num_filters=num_filters_in,
                            kernel_size=1,
                            strides=strides,
                            activation=activation,
                            batch_normalization=batch_normalization,
                            conv_first=False)
            y = resnet_layer(inputs = y,
                            num_filters = num_filters_in,
                            conv_first = False)
            y = resnet_layer(inputs = y,
                            num_filters = num_filters_out,
                            kernel_size = 1,
                            conv_first = False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs = x,
                                num_filters = num_filters_out,
                                kernel_size = 1,
                                strides = strides,
                                activation = None,
                                batch_normalization = False)
            x = keras.layers.add([x, y])
            
        num_filters_in = num_filters_out
    # add classifier on top
    # v2 has BN-ReLU before pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    output = Dense(num_classes, 
                  activation = 'softmax',
                  kernel_initializer = 'he_normal')(y)
    
    # Instantiate model
    model = Model(inputs = inputs, outputs = outputs)
    return model

In [21]:
if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)
    
model.compile(loss = 'categorical_crossentropy',
             optimizer = Adam(lr = lr_schedule(0)),
             metrics = ['accuracy'])
model.summary()
print(model_type)

Instructions for updating:
Colocations handled automatically by placer.
Learning rate:  0.001
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
_______________